In [1]:
import pandas as pd
import numpy as np
from reliability.Repairable_systems import ROCOF, reliability_growth
from reliability.Fitters import Fit_Weibull_2P
from reliability.Probability_plotting import plot_points
from scipy.stats import linregress

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

In [2]:
util_raw = pd.read_csv("Utilization.csv")
util_raw['TO_DATETIME'] = util_raw['TO_DATE'] + ' ' + util_raw['TO_TIME (UTC)']
util_raw['LAND_DATETIME'] = util_raw['LAND_DATE'] + ' ' + util_raw['LAND_TIME (UTC)']
util_raw['FLT_DURR'] = pd.to_datetime(util_raw['LAND_DATETIME']) - pd.to_datetime(util_raw['TO_DATETIME'])
util_raw.drop(columns=['TO_DATE', 'TO_TIME (UTC)', 'LAND_DATE', 'LAND_TIME (UTC)'], inplace=True)
util_raw.head()


,AC_REG,DEP_STA,ARR_STA,CYC,TO_DATETIME,LAND_DATETIME,FLT_DURR
0,PK-PWA,SHJ,CCU,1,4/10/2022 22:43,4/11/2022 3:29,0 days 04:46:00
1,PK-PWA,CCU,CGK,1,4/17/2022 4:56,4/17/2022 10:00,0 days 05:04:00
2,PK-PWA,CGK,KNO,1,NaN,NaN,NaT
3,PK-PWA,KNO,CGK,1,4/24/2022 7:48,4/24/2022 9:58,0 days 02:10:00
4,PK-PWA,CGK,BDJ,1,NaN,NaN,NaT


In [3]:
dr_raw = pd.read_csv("Dispatch Reliability.csv")
dr_raw.head()

,MON-YEAR,MON_NUM_REV_FLIGHTS,MON_NUM_DELAYS,MON_FLEET_HOURS,MON_FLEET_CYC,TOT_FLEET_HOURS,TOT_FLEET_CYC,TOT_TECH_INC
0,2/1/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3/1/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4/1/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5/1/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6/1/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
config_raw = pd.read_csv("AC_CONFIG_CATALOG.csv",)
config_raw.head()

,MSN,REGISTRATION,AIRCRAFT TYPE,AC TOTAL HOURS,...,NLG SHOCK ABSORBER,FORESTAY,LOCK LINK,NLG ACTUATING CYLINDER
0,3809,PK-PWA,A320-214,38400:04,...,D23592020-2,D23596000-3,D23612000,NaN
1,5463,PK-PWC,A320-214,26819:44,...,D23592020-2,D23596000-3,NaN,D2361300


In [5]:
config_raw['AC TOTAL HOURS'] = config_raw['AC TOTAL HOURS'] + ':00'
config_raw['AC TOTAL HOURS'] = pd.to_timedelta(config_raw['AC TOTAL HOURS'])

In [6]:
pwa_tot_util = util_raw[util_raw['AC_REG'] == 'PK-PWA']['FLT_DURR'].sum()
pwc_tot_util = util_raw[util_raw['AC_REG'] == 'PK-PWC']['FLT_DURR'].sum()

pwa_current_total = ((config_raw[config_raw['REGISTRATION'] == 'PK-PWA']['AC TOTAL HOURS'].values + pwa_tot_util)/np.timedelta64(1,'s'))/3600
pwc_current_total = ((config_raw[config_raw['REGISTRATION'] == 'PK-PWC']['AC TOTAL HOURS'].values + pwc_tot_util)/np.timedelta64(1,'s'))/3600

In [7]:
print('The current total hours of PWA: ',pwa_current_total)
print('The current total hours of PWC: ',pwc_current_total)

The current total hours of PWA:  [38419.96666667]
The current total hours of PWC:  [26869.8]


In [8]:
pwa_current_total

array([38419.96666667])